In [9]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy import integrate
from scipy import optimize
from scipy import stats
from scipy import interpolate
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
import quantecon as qecon

In [10]:
def get_labor(v, p, f):
    labor = (v['w']/(p['v']*v['k']**p['theta']*np.exp(v['eps'])))**(1/(p['v']-1))
    return labor

def get_production(v, p, f):
    prod_f = np.exp(v['eps'])*(v['k']**p['theta'])*(f['n'](v, p, f))**p['v']
    return prod_f

def get_objective(v, p, f):
    obj_1 = f['P'](v, p, f) - v['w']*f['n'](v, p, f)
    obj_2 = -(v['k_p'] - (1-p['delta'])*v['k'])
    obj_3 = - (p['phi']/2)*(v['k_p']/v['k'] - (1- p['delta']))**2 *v['k']
    
    obj = obj_1 + obj_2 + obj_3
    return obj

def get_tauchen(rho, sigma_u, m=3, n=7):
    """
    Function found on:
    https://quanteconpy.readthedocs.io/en/0.1.6/_modules/quantecon/tauchen.html

    """
    F = norm(loc=0, scale=sigma_u).cdf

    # standard deviation of y_t
    std_y = np.sqrt(sigma_u**2 / (1-rho**2))

    # top of discrete state space
    x_max = m * std_y

    # bottom of discrete state space
    x_min = - x_max

    # discretized state space
    x = np.linspace(x_min, x_max, n)

    step = (x_max - x_min) / (n - 1)
    half_step = 0.5 * step
    P = np.empty((n, n))

    for i in range(n):
        P[i, 0] = F(x[0]-rho * x[i] + half_step)
        P[i, n-1] = 1 - F(x[n-1] - rho * x[i] - half_step)
        for j in range(1, n-1):
            z = x[j] - rho * x[i]
            P[i, j] = F(z + half_step) - F(z - half_step)

    return x, P

In [11]:
def get_dictionaries():
    p = {'theta':0.21, 'v':0.64, 'delta':0.1, 'beta':0.96, \
         'phi':0.5, 'rho':0.9, 'sigma':0.02, 'n_eps':10}
    f = {'n':get_labor, 'P':get_production, 'obj':get_objective, 'tauchen':get_tauchen}
    
    shocks, transition = f['tauchen'](p['rho'], p['sigma'], n=p['n_eps'])
    mc = qecon.MarkovChain(transition)
    
    v = {'w':0.78297, 'c_f':0.01, 'k_min':0.01, 'c_e':0.02, \
             'k_max':10, 'error':1, 'n':100, 'shocks':shocks, 'transition':transition, \
        'stationary_mc':  mc.stationary_distributions.reshape(1, p['n_eps'])}
    return v, p, f

In [12]:
variables, params, functions = get_dictionaries()

In [13]:
def vfi(v, p, f, thres=1e-3):
    k_grid = np.linspace(v['k_min'], v['k_max'], v['n'])

    V1_old = np.zeros((p['n_eps'], v['n']))
    V1_new = np.zeros_like(V1_old)
    V1_temp = np.zeros_like(V1_new)
    V1 = np.zeros_like(V1_new)
    policy = np.zeros_like(V1_new)
    exit = np.zeros_like(V1_new)+0.5
    diff = np.ones_like(V1_old)
    
    comparison_matrix = ((1-p['delta'])*k_grid).reshape(1, v['n'])
    objective = np.zeros_like(V1_old)
    
    while thres < diff.max():
        v['k_p'] = k_grid.copy()

        for j in range(v['n']):
            v['k'] = k_grid[j]
            for i in range(p['n_eps']):
                v['eps'] = v['shocks'][i]
                objective[i,:] = f['obj'](v, p, f)
            
            V1 = objective + p['beta']*np.dot(v['transition'], V1_old)
            V1_temp[:, j] = V1.max(axis=1)
            policy[:, j] = k_grid[V1.argmax(axis=1)]
        

        V1_new = np.maximum(V1_temp-v['c_f'], comparison_matrix)
        diff = V1_new - V1_old
        diff = linalg.norm(diff)
        V1_old = V1_new.copy()
    
    policy[comparison_matrix >V1_temp-v['c_f']] = 0
    exit[comparison_matrix >V1_temp-v['c_f']] = 1
    exit[comparison_matrix <V1_temp-v['c_f']] = 0
    
    v_hat = (-k_grid +  p['beta']*np.dot(v['stationary_mc'],V1_new)).max()

    return V1_new, v_hat, policy, exit, k_grid

In [14]:
V1_new, v_hat, policy, exit, k_grid = vfi(variables, params, \
                                          functions)

In [15]:
def equations_for_wage(w, v, p, f):
    v['w'] = w
    V1_new, v_hat = vfi(variables, params, functions)[0:2]
    print('Done')
    e = v['c_e'] - (v_hat*v['stationary_mc']).sum()
    return e

def solve_for_wage(v, p, f):
    F = lambda w: equations_for_wage(w, v, p, f)
    w_prime = optimize.root(F, 0.6).x
    return w_prime

In [16]:
optimal_wage = solve_for_wage(variables, params, functions)[0]

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [17]:
print('Optimal wage for the set parameters is:', optimal_wage)

Optimal wage for the set parameters is: 1.3501775291727385
